In [51]:
# coding: utf-8

import sys
from collections import Counter
import re
import random
import urllib
import ef
# you may add imports if needed (and if they are installed)

In [43]:
u = "http://ru.wikipedia.org/?oldid=11298863"
u = "http://lenta.ru/news/2013/12/30/count/?TB_iframe=true&height=650&keepThis=true&width=850"

In [10]:
def extract_features(INPUT_FILE_1, INPUT_FILE_2, OUTPUT_FILE):
    examined = open(INPUT_FILE_1, "r")
    general = open(INPUT_FILE_2, "r")
    result = open(OUTPUT_FILE, "w")

    sample_size = 1000
    min_count = 100

    examined_lines = random.sample(examined.read().split('\n'), sample_size)
    general_lines = random.sample(general.read().split('\n'), sample_size)

    features = extract_features_from_list(examined_lines, general_lines)

    for key, value in features:
        if value < min_count:
            break
        result.write(key + '\t' + str(value) + '\n')

In [11]:
general_files = ["data/urls.lenta.general", "data/urls.wikipedia.general", "data/urls.zr.general"]
examined_files = ["data/urls.lenta.examined", "data/urls.wikipedia.examined", "data/urls.zr.examined"]
res_files = ["results/lenta.res", "results/wikipedia.res", "results/zr.res"]

In [12]:
def extract_features_from_list(QLINK_LIST, UNKNOWN_URLS_LIST):
    lines = []
    features = Counter()

    examined_lines = QLINK_LIST
    general_lines = UNKNOWN_URLS_LIST
    lines.extend(examined_lines)
    lines.extend(general_lines)

    for line in lines:
        if line.find('\n') != -1:
            line = line[0:len(line) - 1]
        features_from_url = extract_features_from_url(line)
        for feature in features_from_url:
            features[feature] += 1

    features = features.most_common()

    return features

In [21]:
def extract_features_from_url(url):
    features = []

    # feature 1
    features.append("segments:" + str(count_segments(url)))
    # feature 2
    for name in extract_param_names(url):
        features.append("param_name:" + name)
    # feature 3
    for param in extract_params(url):
        if len(param) == 1:
            param.append("")
        features.append("param:" + param[0] + "=" + param[1])
    # features 4a - 4f
    segments = extract_segments(url)
    for pos, segment in enumerate(segments):
        segment_decoded = urllib.parse.unquote(segment)
        regex_res = re.findall("[^\\d]+\\d+[^\\d]+$", segment_decoded)
        # feature 4a
        features.append("segment_name_" + str(pos) + ":" + segment)
        # feature 4b
        if segment.isdigit():
            features.append("segment_[0-9]_" + str(pos) + ":1")
        # feature 4c
        if len(regex_res) == 1 and regex_res[0] == segment_decoded:
            features.append("segment_substr[0-9]_" + str(pos) + ":1")
        # feature 4d
        extension = get_extension(segment)
        if len(extension) != 0:
            features.append("segment_ext_" + str(pos) + ":" + extension)
        # feature 4e
        if len(regex_res) == 1 and regex_res[0] == segment_decoded and len(extension) != 0:
            features.append("segment_ext_substr[0-9]_" + str(pos) + ":" + extension)
        # feature 4f
        features.append("segment_len_" + str(pos) + ":" + str(len(segment)))

    return features

In [50]:
b = ef.extract_features(general_files[0], examined_files[0], res_files[0])

In [52]:
a = ef.extract_features_from_url(u)

In [45]:
a

['segments:6',
 'param_name:TB_iframe',
 'param_name:height',
 'param_name:keepThis',
 'param_name:width',
 'param:TB_iframe=true',
 'param:height=650',
 'param:keepThis=true',
 'param:width=85',
 'segment_name_0:news',
 'segment_len_0:4',
 'segment_name_1:2013',
 'segment_[0-9]_1:1',
 'segment_len_1:4',
 'segment_name_2:12',
 'segment_[0-9]_2:1',
 'segment_len_2:2',
 'segment_name_3:30',
 'segment_[0-9]_3:1',
 'segment_len_3:2',
 'segment_name_4:count',
 'segment_len_4:5']

In [46]:
b = ef.extract_features_from_url(u)

In [47]:
b

['segments:6',
 'param_name:TB_iframe',
 'param_name:height',
 'param_name:keepThis',
 'param_name:width',
 'param:TB_iframe=true',
 'param:height=650',
 'param:keepThis=true',
 'param:width=85',
 'segment_name_0:news',
 'segment_len_0:4',
 'segment_name_1:2013',
 'segment_[0-9]_1:1',
 'segment_len_1:4',
 'segment_name_2:12',
 'segment_[0-9]_2:1',
 'segment_len_2:2',
 'segment_name_3:30',
 'segment_[0-9]_3:1',
 'segment_len_3:2',
 'segment_name_4:count',
 'segment_len_4:5']